In [1]:
import sqlite3
from datetime import datetime, date

# SQLite DB 연결
# test.db가 있으면 연결, 없으면 새로 생성
con = sqlite3.connect("PMS_db_car.sqlite",detect_types=sqlite3.PARSE_DECLTYPES)

# Connection 으로부터 Cursor 생성
curs = con.cursor()

# 입차

# 입차 프로세스
1. 차량이 주차장 입구로 들어온다.
2. 카메라가 차량의 번호판을 찍고, 카메라 객체의 인식 모듈이 이 번호판 사진을 인식하여 string으로 리턴한다.
3. 차량번호 string으로 (1) 이 사람이 회원인지 (2) 회원이라면 미납된 요금이 있는지 확인한다.
3-1. 회원인가? 
    YES : car 객체 생성 후 3-2로 
    NO : "회원이 아닙니다." 출력 후 문 열어주지 않는다.
3-2. 미납된 요금이 있는가? 
    YES : 문을 열어준다. 
    NO : 3-2-1로
    3-2-1. 미납 요금 결제를 진행한다. 
        결제 성공 : 문을 열어준다. 
        결제 실패 : "xx원의 미납된 요금이 있습니다." 출력 후 문 열어준다.

# 입차 코드
1. 차가 들어옴
2. 카메라가 번호판 인식
3. 카메라 객체의 인식 모듈이 번호판 인식 후 string 출력
1. 출력된 번호 string 을 가지고 car 객체 car_obj 생성
2. BlockBar.open(car_obj.car_num) 결제 실패 : "xx원의 미납된 요금이 있습니다." 출력 후 문 열어준다.
    --호출--> BlockBar.is_paid(car_obj.car_num) --호출--> BlockBar.is_registered(car_obj.car_num)
    호출된 함수의 리턴값에 따라 최종적으로 BlockBar.open의 리턴값이 true 즉, "열어줌" 또는 flase "닫음" 둘 중 하나로 결정된다.
    BlockBar.is_paid() == false 라면, pay 모듈을 호출하여 pay를 완료한다.
    pay() 결과값 true(미납 요금 없음)이면, 문을 열어준다.
    pay() 결과값 false(여전히 미납 요금이 있음)이면, "xx원의 미납된 요금이 있습니다." 출력 후 문 열어준다.
    

In [15]:
def Camera0_recog():
    return '12가1234'

def BlockBar0_is_registered(car_num):
    return True

def BlockBar0_is_paid(car_num):
    if(BlockBar0_is_registered(car_num)):
        print("회원입니다.")
        
        # 데이터베이스 PARK_PAY 테이블의 car_num 일치하는 튜플 중 Park_is_paid == 0 인 것 있나 확인
        # 있으면, 결제하고 Park_is_paid = 1 로 바꾸고 return True
        # 결제 안 한 것이 여러개면 옛날 것부터 순차적으로 결제
        # 결제 안 된 것 하나라도 있으면 return False
        return True
    
    else:
        print("회원이 아닙니다.")    
        return False     


def BlockBar0_open(car_num):
    if(BlockBar0_is_paid(car_num)):
        print("차단바가 열립니다.")
    else:
        print("차단바를 열 수 없습니다.")

In [20]:
from Car import Car

# 번호판 인식
car_num = Camera0_recog()
print(car_num)

# Car 객체 생성
mycar = Car(car_num)

# 등록된 회원인지 여부 True or False
is_regi = BlockBar0_is_registered(car_num)
print('is_regi : ',is_regi)

# 등록된 회원이면 결제여부 확인, 결제가 모두 완료되어 있으면 차단바 열고 아니면 닫기
BlockBar0_open(mycar.car_num)


12가1234
Car 객체가 생성되었습니다.
Car number = 12가1234
is_regi :  True
회원입니다.
차단바가 열립니다.


# 주차

# 주차 프로세스
1. 전광판이 각 층 별 주차 가능 대수와 전체 주차 가능 대수를 사용자에게 보여준다. 예) B1| 23 / 80 
2. 주차장 칸마다 천장에 설치된 초음파센서가, 보낸 음파 신호와 받은 음파 신호의 시간을 통해 천장으로부터 물체까지의 거리를 측정한다.
    if 천장-물체 거리 < 천장-바닥 거리 (오차범위 있음), 이 칸에 차량이 주차함
    else, 이 칸이 비어 있음
3. 운전자가 주차장 천장에 설치되어 있는 LED를 보고 주차 가능 여부를 확인한다. 
4. 운전자는 초록색 LED로 표시되어 있는 칸으로 이동하여 주차한다. 
5. 주차 후 카메라 센서를 통해 운전자의 차량이 주차를 시작한 시각, 차량 번호가 DB의 PARK_PAY 테이블에 새 튜플로 생성된다. 
6. 주차 후 초음파 센서를 통해 DB의 PARKINGLOT_LIST 테이블에 해당 칸의 주차상태, 해당칸에 주차한 차량번호가 업데이트된다. 
7. 운전자가 주차한 칸 천장의 LED가 빨간색으로 바뀐다.

# 주차 코드
차량이 주차했다는 사실은 카메라 -> Car객체 && 초음파 센서 -> parkinglot 객체 두가지가 인식한다.
1. ParkingLot.get_level_status() 가 전광판에 각 층마다의 주차현황 출력
2. ParkingLot.get_spot_status() 가 LED에 각 칸마다의 주차상태를 색깔로써 출력
3. ParkingLot.proc_usonic() 가 음파 센서와 통신하며 음파 센서의 거리 값에 따라 특정 칸의 주차상태를 실시간으로 출력한다.
4. ParkingLot.updt_status() 는 DB의 PARKINGLOT_LIST 테이블의 특정 칸에 주차상태와 차량번호를 업데이트한다.
5. ParkingLot.detect_change() 가 ParkingLot.proc_usonic() 에서 받은 값에 변동이 있을 때 ParkingLot.set_status() 호출
6. 차량이 LED가 초록색인 주차칸으로 들어온다.
7. 카메라가 차량의 번호판을 찍고, Camera 객체의 인식 모듈이 이 번호판 사진을 인식하여 string으로 리턴한다.
8. 차량번호 string으로 Car 객체를 생성한다. 객체이름은 다른 차량과 겹치지 않게 주차칸이름+차량뒷번호4자리로 한다.
9. Car.wr_parkin() 으로 Car.in_time과 Car.car_num과 Car.location을 DB의 PARK_PAY 테이블에 저장한다. (새 튜플 생성)
10. 차량이 주차했으므로 
10. ParkingLot.updt_status() 가 DB의 PARKINGLOT_LIST 테이블의 특정 칸에 주차상태를 저장한다.
11. ParkingLot.updt_status()으로 DB의 PARKINGLOT_LIST 테이블에서 Car.location과 일치하는 튜플의 Customer_car_num 을 Car.car_num 으로 update 한다. (기존 튜플 업데이트)
12. 

In [25]:
# 각 zone 마다 설치된 카메라.
# zone 의 주차 상태를 알려줌
# Parking_status : 0 == vacant ; 1 == occupied ; 2 == moving_out
def get_pklt_status(zone):
    # zone 에 일치하는 PARKINGLOT_LIST를 return
    '''
    zone = 'B1-A'
    zone_info = []
    cur.execute("select * from PARKINGLOT_LIST where Parking_spot like ?", (zone + '%',))
    rows = cur.fetchall()
    for row in rows:
        zone_info.append(row)
    print(zone_info)
    return zone_info
    '''
    return [['B1-A1', 0, None],['B1-A2', 1, '12가1234'],['B1-A3', 1, '15나1313'],
            ['B1-A4', 2, '23가1122'],['B1-A5', 0, None],['B1-A6', 0, None],['B1-A7', 0, None],
            ['B1-A8', 0, None],['B1-A9', 0, None],['B1-A10', 1,'67마1343']]

def set_pklt_status(zone):
    # zone 에 일치하는 PARKINGLOT_LIST의 status와 차량번호를 update
    # zone 에 일치하는 PARKINGLOT_LIST를 return
    '''
    zone = 'B1-A'

    #### 변경 전 ####
    print('******** 변경 전 ********')
    zone_info = []
    cur.execute("select * from PARKINGLOT_LIST where Parking_spot like ?", (zone + '%',))
    rows = cur.fetchall()
    for row in rows:
        zone_info.append(list(row))
    for row in zone_info:
        print(row)

        
    #### 변경 중 ####
    zone_info[0][1] = 1
    zone_info[0][2] = '49루0663'
    # UPDATE table이름 SET 컬럼이름 = 변경된값 WHERE 특정튜플
    cur.execute("UPDATE PARKINGLOT_LIST SET Parking_status = ?, Customer_car_num = ? WHERE Parking_spot = ?",
                (zone_info[0][1], zone_info[0][2], zone_info[0][0],))


    #### 변경 후 ####
    print('******** 변경 후 ********')
    cur.execute("select * from PARKINGLOT_LIST where Parking_spot like ?", (zone + '%',))
    zone_info_after = []
    rows = cur.fetchall()
    for row in rows:
        zone_info_after.append(list(row))
    for row in zone_info_after:
        print(row)
    #return zone_info
    
    '''
    return [['B1-A1', 1, '49루0663'],['B1-A2', 1, '12가1234'],['B1-A3', 1, '15나1313'],
            ['B1-A4', 2, '23가1122'],['B1-A5', 0, None],['B1-A6', 0, None],['B1-A7', 0, None],
            ['B1-A8', 0, None],['B1-A9', 0, None],['B1-A10', 1,'67마1343']]

def 

In [30]:
zone_B1A = Parkinglot_status('B1-A')
# 전체 배열
for spot in zone_B1A:
    print(spot)
print('********')

# 'B1-A1'의 정보
print(zone_B1A[0])
# 'B1-A1'
print(zone_B1A[0][0])
# 'B1-A1'의 status
print(zone_B1A[0][1])
# 'B1-A1'에 주차한 차 번호
print(zone_B1A[0][2])

['B1-A1', 0, None]
['B1-A2', 1, '12가1234']
['B1-A3', 1, '15나1313']
['B1-A4', 2, '23가1122']
['B1-A5', 0, None]
['B1-A6', 0, None]
['B1-A7', 0, None]
['B1-A8', 0, None]
['B1-A9', 0, None]
['B1-A10', 1, '67마1343']
********
['B1-A1', 0, None]
B1-A1
0
None


In [ ]:
# 차량이 주차하면 Parkinglot 상태가 변경됨
# 변경되는 것은 센서를 통해 감지
set_Parkinglot(zone)

